In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy
import cv2

In [2]:
#Setting the directories
train_dir = 'D:/DeepLearningSpecialization/Assignment1LRCatDogs/forTrain/'
test_dir = 'D:/DeepLearningSpecialization/Assignment1LRCatDogs/forTest/'

# setting up paths for each images in two list test and train
train_images = [train_dir + train_img for train_img in os.listdir(train_dir)]
test_images =  [test_dir + test_img for test_img in os.listdir(test_dir)]

In [4]:
def Read_Image(image):
    image = cv2.imread(image, cv2.IMREAD_COLOR)
    return cv2.resize(image,(64,64),interpolation = cv2.INTER_CUBIC)

In [9]:
def Create_Data(images):
    m = len(images)
    
    X = np.zeros((m,64,64,3),dtype=np.uint8)
    y = np.zeros((1,m))
    
    for index, image in enumerate(images):
        X[index,:] = Read_Image(image)
        
        y[0,index] = 1 if 'dog' in image.lower() else 0
    
    return X,y

In [10]:
train_set_x , train_set_y = Create_Data(train_images)
test_set_x, test_set_y = Create_Data(test_images)

In [11]:
train_set_x = train_set_x.reshape(train_set_x.shape[0], 3*64*64).T
test_set_x = test_set_x.reshape(test_set_x.shape[0], -1).T

In [12]:
train_set_x = train_set_x / 255  #Broadcasting concept here
test_set_x  = test_set_x / 255 

In [14]:
train_set_x.shape

(12288, 412)

In [15]:
train_set_y.shape

(1, 412)

### Layers

In [32]:
def layer_sizes(X, y):
    
    number_inputs = X.shape[0]
    hidden_layers = 4
    number_output = y.shape[0]
    return (number_inputs, hidden_layers, number_output)

In [18]:
def initialize_parameters(n_x, n_h, n_y):
    
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros(shape=(n_y, 1))
    
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [21]:
parameters = initialize_parameters(train_set_x.shape[0], 4, train_set_y.shape[0])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[-0.00416758 -0.00056267 -0.02136196 ... -0.01275513  0.00733372
   0.01166127]
 [ 0.00016076  0.00699494 -0.02144254 ... -0.00569705 -0.00276633
   0.0011016 ]
 [-0.00553107  0.00678748 -0.00870469 ...  0.00174631  0.00395599
  -0.00136441]
 [-0.00701089  0.00335768 -0.00122743 ...  0.01439597 -0.00733648
  -0.0040589 ]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01336063  0.00115223 -0.00207114  0.01109624]]
b2 = [[0.]]


In [22]:
train_set_x.shape

(12288, 412)

In [25]:
def forward_propagation(X, parameters):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [26]:
def compute_cost(A2, Y, parameters):
   
    
    m = Y.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    logprobs = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))
    cost = - np.sum(logprobs) / m
    
    cost = np.squeeze(cost)
    
    return cost

In [27]:
def backward_propagation(parameters, cache, X, Y):

    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### (≈ 2 lines of code)
    W1 = parameters['W1']
    W2 = parameters['W2']
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### (≈ 2 lines of code)
    A1 = cache['A1']
    A2 = cache['A2']
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### (≈ 6 lines of code, corresponding to 6 equations on slide above)
    dZ2= A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [28]:
def update_parameters(parameters, grads, learning_rate=1.2):
    
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### (≈ 4 lines of code)
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [34]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [29]:
def nn_model(X, Y, n_h, num_iterations=10000, print_cost=False):
   
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]

    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']


    for i in range(0, num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y, parameters)
 
       
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grads)
        
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))

    return parameters

In [30]:
def predict(parameters, X):
    
    A2, cache = forward_propagation(X, parameters)
    predictions = np.round(A2)
    
    return predictions

In [ ]:
parameters = nn_model(train_set_x, train_set_y, n_h = 4, num_iterations=10000, print_cost=True)

Cost after iteration 0: 0.688188
Cost after iteration 1000: 0.000166
Cost after iteration 2000: 0.000083
Cost after iteration 3000: 0.000056
Cost after iteration 4000: 0.000042
Cost after iteration 5000: 0.000033
Cost after iteration 6000: 0.000028
Cost after iteration 7000: 0.000024
Cost after iteration 8000: 0.000021
Cost after iteration 9000: 0.000019
